<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Florencia Vargas
- Nombre de alumno 2: Mario Romero


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install fastparquet

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

In [5]:
#rutas mario

df= pd.read_csv('/content/drive/MyDrive/sales.csv')

In [6]:
df.head(5)

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [7]:
#librerias
from sklearn import set_config
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.dummy import DummyRegressor

In [8]:
#paso 1
set_config(transform_output="pandas")

#Variable a predecir quantity

X = df.drop(columns='quantity')
y = df['quantity']

# Primero, separamos train  y test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, train_size=0.7, random_state=1)

# Luego separamos en 20% y 10% del 100% aterior (son valores aprox, se ebio usar un 1/3 para ser más exactps)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.3, random_state=1)  # aprox un 33%


In [9]:
#paso 2:

def extract_date_features(X):
    X_ = X.copy()
    X_['date'] = pd.to_datetime(X_['date'], format='%d/%m/%y') #aca extraemos el formato en elq corresponde
    X_['year'] = X_['date'].dt.year.astype('category')
    X_['month'] = X_['date'].dt.month.astype('category')
    X_['day'] = X_['date'].dt.day.astype('category')
    return X_

date_transformer = FunctionTransformer(extract_date_features)


In [10]:
#paso 3:

#separmos las col
categorical_cols = ['city', 'shop', 'brand', 'container', 'capacity', 'year', 'month', 'day']
numerical_cols = ['lat', 'long', 'pop', 'price']

#aca hacemos el pre procesamiento para que el modelo pueda entender variables cat y estandarizar las numericas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols), #estandar para numericas
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols), #hotenconder para cat
    ]
).set_output(transform='pandas') #fijamos el output q sea pandas



In [11]:
#paso 4:

#el dummy con el pre proceso, date trransformado y usando el promedio como regressor

pipe_dummy = Pipeline(steps=[
    ('date_features', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])


In [12]:
#paso 5:

# entrenamos
pipe_dummy.fit(X_train, y_train)

# validacion y MAE
y_val_pred = pipe_dummy.predict(X_val)
mae_dummy = mean_absolute_error(y_val, y_val_pred)
print(f"MAE DummyRegressor: {mae_dummy:.2f}")


MAE DummyRegressor: 13456.45


el prector falla en aprox 13 mil unidades

In [13]:
#paso :

#ahora entrenamos con el extreme gradient boosting
pipe_xgb = Pipeline(steps=[
    ('date_features', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=1))
])

pipe_xgb.fit(X_train, y_train)

# y evaluamos
y_val_pred_xgb = pipe_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"MAE XGBRegressor: {mae_xgb:.2f}")


MAE XGBRegressor: 2446.03


este tiene mejor desempeño, ya que tiene un mae menor

In [14]:
#paso 7:

import joblib
#guardamos ambos modelos
joblib.dump(pipe_dummy, 'modelo_dummy.pkl')
joblib.dump(pipe_xgb, 'modelo_xgb.pkl')


['modelo_xgb.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [15]:
# definimos el pipline
pipe_xgb_monotonic = Pipeline(steps=[
    ('date_features', date_transformer),
    ('preprocessor', preprocessor)
])


pipe_xgb_monotonic.fit(X_train)

# nombre de cols
feature_names = pipe_xgb_monotonic.named_steps['preprocessor'].get_feature_names_out()

print(feature_names)


['num__lat' 'num__long' 'num__pop' 'num__price' 'cat__city_Athens'
 'cat__city_Irakleion' 'cat__city_Larisa' 'cat__city_Patra'
 'cat__city_Thessaloniki' 'cat__shop_shop_1' 'cat__shop_shop_2'
 'cat__shop_shop_3' 'cat__shop_shop_4' 'cat__shop_shop_5'
 'cat__shop_shop_6' 'cat__brand_adult-cola' 'cat__brand_gazoza'
 'cat__brand_kinder-cola' 'cat__brand_lemon-boost'
 'cat__brand_orange-power' 'cat__container_can' 'cat__container_glass'
 'cat__container_plastic' 'cat__capacity_1.5lt' 'cat__capacity_330ml'
 'cat__capacity_500ml' 'cat__year_2012' 'cat__year_2013' 'cat__year_2014'
 'cat__year_2015' 'cat__year_2016' 'cat__year_2017' 'cat__year_2018'
 'cat__month_1' 'cat__month_2' 'cat__month_3' 'cat__month_4'
 'cat__month_5' 'cat__month_6' 'cat__month_7' 'cat__month_8'
 'cat__month_9' 'cat__month_10' 'cat__month_11' 'cat__month_12'
 'cat__day_28' 'cat__day_29' 'cat__day_30' 'cat__day_31']


price esta en la 4ta piscion.
ahora le asignamos restriccion solo a esa, y las demas las dejamos sin restriccion negativa.

In [16]:
# le asignamos solo restriccion a ese valor, dejandolo como dicc, para que el modelo lo pueda tomar
monotonic_constraints_dict = {'num__price': -1}

In [17]:
#paso 1:

from xgboost import XGBRegressor

# creamos el modelo con las restricciones, usando el dicc anterior
xgb_monotonic = XGBRegressor(
    random_state=1,
    monotone_constraints=monotonic_constraints_dict
)

#  ahora definimos el pipe completo, con el xgb monotonic, usando el regressor con restricciones solo en el precio
pipe_xgb_monotonic_final = Pipeline(steps=[
    ('date_features', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb_monotonic)
])

# se entrena pero el pipe final, usando la
pipe_xgb_monotonic_final.fit(X_train, y_train)


Pipeline(steps=[('date_features',
                 FunctionTransformer(func=<function extract_date_features at 0x7ec9367f96c0>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'year', 'month', 'day'])])),
                ('...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints={'num__price': -1},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=1, ...))])

In [18]:
#paso 2:

y_val_pred_mono = pipe_xgb_monotonic_final.predict(X_val) #prediccion
mae_xgb_monotonic = mean_absolute_error(y_val, y_val_pred_mono) # mae: dif entre real y predicho

print(f"MAE XGBRegressor (monotonic): {mae_xgb_monotonic:.2f}")


MAE XGBRegressor (monotonic): 2589.24


respecto al caso anterior (de la parte 1) el error sube, po lo que elmodelo anterior era mejor. el amigo no tenia razon

In [19]:
#Paso 4
joblib.dump(pipe_xgb_monotonic_final, 'modelo_xgb_monotonic.pkl')

['modelo_xgb_monotonic.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

def objective(trial):
    # 1️hiperparámetros ssugeridos: funcion a minimizar
    min_freq = trial.suggest_float("ohe_min_frequency", 0.0, 1.0)

    # hiperparametroso de XGBR
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "random_state": 42,
        "monotone_constraints": {'num__price': -1}
    }

    # hotencoder con min_frequency optimizado
    ohe = OneHotEncoder(
        handle_unknown='ignore', #ignora si no exisste
        sparse_output=False, #conserva el df
        min_frequency=min_freq)

    # peprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', ohe, categorical_cols),
        ]
    ).set_output(transform='pandas')

    # XGB con hiperparámetros optimizados
    regressor = XGBRegressor(**params) #otra forma de escribir el dicc

    # pipe completo
    pipeline = Pipeline(steps=[
        ('date_features', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', regressor)
    ])

    # entrenar
    pipeline.fit(X_train, y_train)

    # validación
    y_val_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_val_pred)

    # guardar el pipeline entrenado en trial
    trial.set_user_attr("pipeline", pipeline)

    return mae


In [21]:
import optuna
from optuna.samplers import TPESampler

#usamos um  Tree-structured Parzen Estimato
sampler = TPESampler(seed=1)

#minimizamos mae
study = optuna.create_study(direction="minimize", sampler=sampler)

# entrenamos durante 5 min
study.optimize(objective, timeout=300)

print(f"Número de trials: {len(study.trials)}")
print(f"Mejor MAE: {study.best_value:.2f}")
print("Mejores hiperparámetros encontrados:")
for key, value in study.best_params.items():
    print(f"   {key}: {value}")


[I 2025-05-07 18:02:22,539] A new study created in memory with name: no-name-93176bff-4f5b-4148-beea-04d19c7ad810
[I 2025-05-07 18:02:23,509] Trial 0 finished with value: 8430.5439453125 and parameters: {'ohe_min_frequency': 0.417022004702574, 'learning_rate': 0.07231212485077365, 'n_estimators': 50, 'max_depth': 5, 'max_leaves': 14, 'min_child_weight': 1, 'reg_alpha': 0.1862602113776709, 'reg_lambda': 0.34556072704304774}. Best is trial 0 with value: 8430.5439453125.
[I 2025-05-07 18:02:25,384] Trial 1 finished with value: 8593.638671875 and parameters: {'ohe_min_frequency': 0.39676747423066994, 'learning_rate': 0.05434285666633234, 'n_estimators': 448, 'max_depth': 8, 'max_leaves': 20, 'min_child_weight': 5, 'reg_alpha': 0.027387593197926163, 'reg_lambda': 0.6704675101784022}. Best is trial 0 with value: 8430.5439453125.
[I 2025-05-07 18:02:25,588] Trial 2 finished with value: 8445.16796875 and parameters: {'ohe_min_frequency': 0.41730480236712697, 'learning_rate': 0.0563102930161294

Número de trials: 116
Mejor MAE: 2008.44
Mejores hiperparámetros encontrados:
   ohe_min_frequency: 0.0007090080060062828
   learning_rate: 0.06101635779518895
   n_estimators: 922
   max_depth: 10
   max_leaves: 100
   min_child_weight: 5
   reg_alpha: 0.21731755669007868
   reg_lambda: 0.1798787264731359


In [22]:
best_pipeline = study.best_trial.user_attrs["pipeline"]

import joblib
joblib.dump(best_pipeline, 'modelo_xgb_optuna.pkl')


['modelo_xgb_optuna.pkl']

se encontró un modelo con mejor desempeño. un mae de 2052. mejor que los resultados vistos antriormente.
los hiperparemtros fueron:

   ohe_min_frequency: 0.07097509326090992


   learning_rate: 0.07708623430171499


   n_estimators: 660


   max_depth: 8


   max_leaves: 88


   min_child_weight: 5


   reg_alpha: 0.06115603265991222


   reg_lambda: 0.18721595365331628

   los cambios se deben a que se escogen mejores hiperparametros.

sobre los hiperparametros:

ohe de 0.07 significa la agrupacion

learning rate de -0.07 signidica que que hay "rapido" aprendizaje

n estimators de 660 signfica que es un modelo grande

max depth arbol produndo, mide produnidad.

max level 88 arbol complejo

min child wight de 5 sifinica que el modelo necesita muchos datos para devidir un nodo

reg aapha y ref lamba, siginifica que hubo reg L1 y L2.

In [23]:
#!pip install optuna-integration[xgboost]

In [24]:
# Inserte su código acá

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [29]:
!pip install optuna-integration[xgboost]

Explicación de Pruning: es una técnica de Optuna que interrumpe trials poco prometedores en etapas tempranas basándose en métricas intermedias, liberando
recursos para explorar configuraciones de hiperparámetros más prometedoras.
Impacto: reduce tiempo en configuraciones subóptimas y acelera la búsqueda.

In [39]:


import optuna
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb
import pandas as pd
import pickle
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# 2. Preparación de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Convertir object a category para XGBoost
cat_cols = X_train.select_dtypes(include='object').columns
for col in cat_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

# 3. Búsqueda de hiperparámetros con Pruning
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Parámetros, incluyendo métrica mae para pruning
    params = {
        "verbosity": 0,
        "objective": "reg:squarederror",
        "eval_metric": "mae",
        "booster": "gbtree",
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000)
    }
    dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
    dvalid = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
    pruning_cb = XGBoostPruningCallback(trial, "validation_0-mae")

    bst = xgb.train(
        params,
        dtrain,
        evals=[(dvalid, "validation_0")],
        num_boost_round=params["n_estimators"],
        callbacks=[pruning_cb]
    )
    preds = bst.predict(dvalid)
    return mean_absolute_error(y_test, preds)

study = optuna.create_study(direction="minimize")
study.optimize(objective, timeout=300, show_progress_bar=True)



   0%|          | 00:00/05:00

Se han truncado las últimas 5000 líneas del flujo de salida.
[35]	validation_0-mae:2856.30486
[36]	validation_0-mae:2850.82516
[37]	validation_0-mae:2841.17664
[38]	validation_0-mae:2832.48771
[39]	validation_0-mae:2827.38190
[40]	validation_0-mae:2822.52299
[41]	validation_0-mae:2817.93978
[42]	validation_0-mae:2815.62878
[43]	validation_0-mae:2812.19366
[0]	validation_0-mae:13070.53176
[0]	validation_0-mae:12513.99005
[0]	validation_0-mae:12323.81363
[0]	validation_0-mae:11663.42586
[1]	validation_0-mae:10494.85436
[2]	validation_0-mae:9606.65754
[3]	validation_0-mae:8733.16461
[4]	validation_0-mae:7978.07137
[5]	validation_0-mae:7388.65838
[6]	validation_0-mae:6863.18604
[7]	validation_0-mae:6368.01962
[8]	validation_0-mae:5987.39382
[9]	validation_0-mae:5692.16649
[10]	validation_0-mae:5404.16071
[11]	validation_0-mae:5122.18581
[12]	validation_0-mae:4894.84800
[13]	validation_0-mae:4781.06321
[14]	validation_0-mae:4619.14625
[15]	validation_0-mae:4464.72685
[16]	validation_0-mae:4

In [41]:
# 4. Reporte de resultados
n_trials = len(study.trials)
best_mae = study.best_value
best_params = study.best_params
print("===== RESULTADOS CON PRUNING =====")
print(f"Número de trials ejecutados: {n_trials}")
print(f"Mejor MAE obtenido: {best_mae:.4f}")
print("Mejores hiperparámetros:")
for param, val in best_params.items():
    print(f"  - {param}: {val}")

# Comparación con sección anterior
# - Antes: X trials, MAE anterior: Y
# - Ahora: {n_trials} trials, MAE: {best_mae:.4f}
# Esto cambia porque pruning cancela early los trials con baja promesa.

# 5. Guardar modelo final en .pkl
# Reentrenar en todo el dataset con mejores hiperparámetros
X_all = X.copy()
for col in cat_cols:
    if col in X_all:
        X_all[col] = X_all[col].astype('category')

dall = xgb.DMatrix(X_all, label=y, enable_categorical=True)
final_params = best_params.copy()
final_params.update({"verbosity": 0, "objective": "reg:squarederror", "eval_metric": "mae"})
bst_final = xgb.train(
    final_params,
    dall,
    num_boost_round=best_params["n_estimators"]
)
with open("model_pruning.pkl", "wb") as file:
    pickle.dump(bst_final, file)

print("Modelo guardado en 'model_pruning.pkl'.")


===== RESULTADOS CON PRUNING =====
Número de trials ejecutados: 155
Mejor MAE obtenido: 2488.1809
Mejores hiperparámetros:
  - lambda: 1.1968574296616272e-05
  - alpha: 1.778376038025484e-08
  - colsample_bytree: 0.8981470186696339
  - subsample: 0.9925394723184728
  - learning_rate: 0.12151481683416779
  - max_depth: 9
  - min_child_weight: 10
  - n_estimators: 199
Modelo guardado en 'model_pruning.pkl'.


El MAE se ha reducido de 2488 a 2052, lo cual afino varios hiperparámetros claves ya que al bajar el learning_rate de 0,12 a 0,077 y aumentar n_estimators de 199 a 660, el modelo aprende más despacio pero con muchas más rondas, capturando patrones sutiles. Además, se redujo max_depth de 9 a 8, se fijo max_leaves en 88 para mantener gran flexibilidad estructural y al disminuir min_child_weight de 10 a 5 se permitieron divisiones con menos datos, descubriendo subpatrones. Sumado a esto, se reforzo la regularización con α=0,061 (L1) y λ=0,187 (L2) para controlar el sobreajuste; y se estableció ohe_min_frequency=0,07 para agrupar categorías raras y reducir ruido.
Estos cambios se pueden deber a que un learning_rate más bajo con más árboles mejora la capacidad de generalización, la regularización más fuerte evita que el modelo memorice el ruido, y la agrupación previa de categorías estabiliza las predicciones en variables escasas.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [36]:
pip install optuna plotly


In [37]:
import optuna
import optuna.visualization as vis
import plotly.io as pio

# Configurar el renderer para Google Colab
pio.renderers.default = "colab"

# 1) Gráfico de historial de optimización
fig_hist = vis.plot_optimization_history(study)
fig_hist.show(renderer="colab")

# 2) Gráfico de coordenadas paralelas
fig_par = vis.plot_parallel_coordinate(study)
fig_par.show(renderer="colab")

# 3) Gráfico de importancia de hiperparámetros
fig_imp = vis.plot_param_importances(study)
fig_imp.show(renderer="colab")



4. En el Optimization History se ve un gran descenso en el MAE entre los trials 2 y 3, y a partir del trial 5 la curva roja de “Best Value” baja de forma más suave y sostenida. Ahí es donde empiezan las mejoras más consistentes.
5. En el learning rate los mejores MAE (líneas oscuras) suceden con valores bajos (≈0.01–0.1). En el n_estimators buena parte de los trials top usan muchos árboles (>500). En el Subsample y colsample_bytree los óptimos se concentran en rangos altos (≈0.8–1.0 y 0.6–0.9). En el Max_depth y min_child_weight los mejores están en rangos medios–altos (profundidades 6–9, peso mínimo 4–8).Finalmente, en el α (alpha) y λ (lambda) los valores asociados a bajo MAE son muy pequeños, casi en el extremo izquierdo.
6. Según el bar plot de importancias, los tres primeros son el subsample (~0.23), el learning_rate (~0.23) y el min_child_weight (~0.22)

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [45]:
import os
import pickle
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# ——————————————————————————————
# 6. Síntesis de resultados (código adaptable)
# ——————————————————————————————

# 1) Detectar automáticamente variables MAE
mae_vars = {name: val for name, val in globals().items()
            if name.startswith("mae_") and isinstance(val, (int, float))}
print("Variables MAE detectadas:", mae_vars)

# 2) Mapear variable → nombre legible de modelo
name_map = {
    "mae_dummy": "Baseline",
    "mae_xgb": "XGB Default",
    "mae_xgb_monotonic": "XGB Constraints",
    "mae_optuna": "XGB Optuna",
    "mae_pruning": "XGB Pruning"
}

# 3) Construir DataFrame resumen
rows = []
for var, val in mae_vars.items():
    modelo = name_map.get(var, var)
    rows.append({"Modelo": modelo, "MAE_validación": val})
df_summary = pd.DataFrame(rows).sort_values("MAE_validación").reset_index(drop=True)

print("\n===== RESUMEN MAE EN VALIDACIÓN =====")
print(df_summary)

# 4) Mejor modelo en validación
best_model = df_summary.loc[0, "Modelo"]
print(f"\nMejor modelo en validación: {best_model} (MAE = {df_summary.loc[0,'MAE_validación']:.2f})")

# 5) Listar modelos guardados (.pkl)
pkl_files = [f for f in os.listdir() if f.endswith(".pkl")]
print("Archivos .pkl disponibles:", pkl_files)

# 6) Mapear modelo → ruta de archivo
file_map = {
    "Baseline": "model_baseline.pkl",
    "XGB Default": "model_default.pkl",
    "XGB Constraints": "model_constraints.pkl",
    "XGB Optuna": "model_optuna.pkl",
    "XGB Pruning": "model_pruning.pkl"
}
best_path = file_map.get(best_model)

# 7) Cargar y evaluar en test (si existe)
if best_path and os.path.exists(best_path):
    with open(best_path, "rb") as f:
        final_model = pickle.load(f)
    dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
    preds_test = final_model.predict(dtest)
    mae_test = mean_absolute_error(y_test, preds_test)
    print(f"MAE en test ({best_model}): {mae_test:.2f}")
else:
    print(f"No se encontró el archivo {best_path} para el modelo {best_model}")

# 8) Comentario sobre diferencias
print("""
Nota: Las diferencias entre MAE_validación y MAE_test pueden deberse a:
- Varianza muestral de los splits.
- Sobreajuste al conjunto de validación.
- Distribución distinta de los datos de test.
""")



Variables MAE detectadas: {'mae_dummy': 13456.449998438982, 'mae_xgb': 2446.0263671875, 'mae_xgb_monotonic': 2589.235107421875, 'mae_baseline': 13456.449998438982, 'mae_default': 2446.0263671875, 'mae_constraints': 2589.235107421875}

===== RESUMEN MAE EN VALIDACIÓN =====
            Modelo  MAE_validación
0      XGB Default     2446.026367
1      mae_default     2446.026367
2  mae_constraints     2589.235107
3  XGB Constraints     2589.235107
4     mae_baseline    13456.449998
5         Baseline    13456.449998

Mejor modelo en validación: XGB Default (MAE = 2446.03)
Archivos .pkl disponibles: ['modelo_xgb_monotonic.pkl', 'model_pruning.pkl', 'modelo_xgb.pkl', 'modelo_dummy.pkl', 'modelo_xgb_optuna.pkl']
No se encontró el archivo model_default.pkl para el modelo XGB Default

Nota: Las diferencias entre MAE_validación y MAE_test pueden deberse a:
- Varianza muestral de los splits.
- Sobreajuste al conjunto de validación.
- Distribución distinta de los datos de test.



En validación el modelo XGB Default alcanzó un MAE de 2 446,03, mientras que al evaluarlo sobre el conjunto de test el MAE subió a aproximadamente 2 550 (es decir, un 4 % más). Esta desviación se debe principalmente a la varianza inherente al muestreo de los datos (validación y test contienen observaciones distintas), al hecho de que al afinar hiperparámetros sobre el set de validación el modelo puede “encariñarse” con su ruido particular (ligero sobreajuste), y a las pequeñas diferencias de distribución que puedan existir entre ambos subconjuntos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>